In [28]:
import s3fs
import numpy as np
import xarray as xr
import fsspec
import zarr
import timeit
import matplotlib.pyplot as plt
import statistics
import gc
from dask.distributed import Client

In [3]:
URL = 's3://mur-sst/zarr'

In [6]:
start_date = "2002-06-01"
end_date = "2002-12-31"

minlat = 18
maxlat = 25
minlon = -160
maxlon = -150

In [4]:
mur_global = xr.open_zarr(
    fsspec.get_mapper(URL, anon=True), 
    consolidated=True, 
)

In [30]:
variables=[
    'analysed_sst', 
    'mask'
]

mur_hawaii = mur_global[variables].sel(
    time=slice(start_date, end_date),
    lat=slice(minlat, maxlat), 
    lon=slice(minlon, maxlon),
).chunk({"time": 10, "lat": 100, "lon": 100})

In [31]:
mur_hawaii_sst = mur_hawaii['analysed_sst'].where(mur_hawaii.mask == 1)

In [32]:
mur_hawaii_sst = mur_hawaii_sst - 273.15

In [33]:
mur_hawaii_sst

,Array,Chunk
Bytes,600.66 MB,400.00 kB
Shape,"(214, 701, 1001)","(10, 100, 100)"
Count,153318 Tasks,1936 Chunks
Type,float32,numpy.ndarray


### For Loop for Finding Daily Mean Temperature

In [ ]:
start_time = timeit.default_timer()

mur_hawaii_sst[0].mean(["lat", "lon"]).values

elapsed = timeit.default_timer() - start_time
print(elapsed)

In [23]:
mur_hawaii_sst_mean_ts = []

for day in range(0, len(mur_hawaii_sst["time"])): 
    print(day)
    mur_hawaii_sst_day_mean = mur_hawaii_sst[day].mean(["lat", "lon"]).values
    mur_hawaii_sst_mean_ts.append(mur_hawaii_sst_day_mean)

    del mur_hawaii_sst_day_mean
    gc.collect()

0


KeyboardInterrupt: 

In [ ]:
mur_hawaii_sst_mean_ts

### For Loop for Finding Mean Temperature Per Coordinate Pair

In [35]:
mur_hawaii_sst[0][18][-160].compute()

<xarray.DataArray 'analysed_sst' ()>
array(25.075989, dtype=float32)
Coordinates:
    lat      float32 18.18
    lon      float32 -151.6
    time     datetime64[ns] 2002-06-01T09:00:00

In [27]:
mur_hawaii_sst_day = []
mur_hawaii_sst_mean_ts = []

for day in range(0, len(mur_hawaii_sst["time"])): 
    print(day)
    for lat in range(0, len(mur_hawaii_sst["lat"])): 
        for lon in range(0, len(mur_hawaii_sst["lon"])):
            mur_hawaii_sst_coord = mur_hawaii_sst[day][lat][lon].values
            mur_hawaii_sst_day.append(mur_hawaii_sst_coord)
            
            del mur_hawaii_sst_coord
            gc.collect()
    mur_hawaii_sst_mean_ts.append(mean(mur_hawaii_sst_day))
    mur_hawaii_sst_day.clear()

0


NameError: name 'gc' is not defined

### Plotting

In [ ]:
mur_hawaii_sst_mean_ts = mur_hawaii_sst.mean(['lat', 'lon'])

In [ ]:
mur_hawaii_sst_mean_sp = mur_hawaii_sst.mean(['time'])